In [13]:
import os 
import pandas as pd
import numpy as np

pathTesis = os.getcwd()
pathBases = os.path.join(pathTesis,'bases/DatosPrecia/PreciosConColumnas')
pathYield = os.path.join(pathTesis,'bases/DatosPrecia/yield')
pathMaturity = os.path.join(pathTesis,'bases/DatosPrecia/tablaMaturity')

In [ ]:
for nom in os.listdir(pathBases):
    nomSinCSV = nom.replace('.csv','')
    
    ### Leo el archivo entregado por precia pero con columnas
    data = pd.read_csv(os.path.join(pathBases,nom),sep='|')

    ### Creo un nuevo dataframe solo con las siguientes columnas
    nemotecnicos = data[['Nemotecnico','DiasVencimiento']]

    ### Agrupo por nemotecnico y creo una nuevacolumna con el maximo de dias por nemotecnico
    maturity = (nemotecnicos.groupby(['Nemotecnico'])['DiasVencimiento'].max()/30).\
                                    reset_index().sort_values('DiasVencimiento').reset_index(drop=True)

    ### Cambio los nombres    
        
    maturity.columns = ['Nemotecnico','vencimientoMeses']

    ### Redondeo hacia arriba
    
    maturity.loc[:,'vencimiento'] = np.floor(maturity.loc[:,'vencimientoMeses']) 

    ### Creo una nueva columna si es menor a 12 entonces es mes y si no es an\o
    
    maturity.loc[:,'periodo'] = np.where(maturity.loc[:,'vencimientoMeses']<12,'M','Y')

    ### Si es an\o dividalo entre 12 sino dejelo igual
    

    maturity.loc[:,'vencimiento'] = np.where(maturity.loc[:,'periodo'] == 'Y',
                                             np.floor(maturity.loc[:,'vencimiento'] / 12),
                                             maturity.loc[:,'vencimiento'])

    ### Vuelvo el vencimiento un entero
    
    maturity.loc[:,'vencimiento'] = maturity.loc[:,'vencimiento'].astype('int')

    ### Concateno las dos columnas vencimiento y periodo

    maturity.loc[:,'maturity'] = [str((maturity.loc[i,'vencimiento'])) + maturity.loc[i,'periodo'] for i in range(len(maturity))]

    ### Creo un nuevo dataframe con el maturity el nemotecnico y el vencimiento en numero de meses
    
    tablaMaturity = maturity[['maturity','Nemotecnico','vencimientoMeses']]

    nemosOriginal = tablaMaturity.loc[:,'Nemotecnico']
    
    ### Se calcula el renimiento para cada uno de los nemotecnicos y se pivotea

    dataNemos = data[data.loc[:,'Nemotecnico'].isin(nemosOriginal)].reset_index(drop=True)[['Nemotecnico','Fecha','Precio','PrecioAnterior','Spread','PrecioLimpio','DiasVencimiento']]
    
    dataMaturity = pd.merge(dataNemos,tablaMaturity,on = ['Nemotecnico'], how = 'left').sort_values(['vencimientoMeses','Fecha']).reset_index(drop=True)

    dataMaturity.loc[:,'yield'] = dataMaturity.eval("((Spread + ((100 - PrecioLimpio)/(DiasVencimiento/365)))/((100 + PrecioLimpio)/2))*100")

    dataMaturity.loc[:,'yield'] = np.round(dataMaturity.loc[:,'yield'],2)
    
    dataMaturity_0 = dataMaturity[['Fecha','Nemotecnico','yield']]

    listaMaturity = list(tablaMaturity.loc[:,'Nemotecnico'].values)

    dataMaturity = pd.pivot_table(dataMaturity,values = 'yield',index = ['Fecha'], columns = ['Nemotecnico'] ).reset_index()[['Fecha']+listaMaturity]
    dataMaturity = pd.DataFrame(dataMaturity.values,columns= ['Fecha']+listaMaturity)
    
    dataMaturity.to_csv(os.path.join(pathYield,nomSinCSV + '_yield.csv'),sep='|',index=False)
    tablaMaturity.to_csv(os.path.join(pathMaturity,nomSinCSV + '_mat.csv'),sep='|',index=False)